In [24]:
from typing import Iterator

class Coordinate(object):
    def __init__(self, i: int, j: int, k: int):
        self.i = i
        self.j = j
        self.k = k

    @staticmethod
    def zero() -> "Coordinate":
        return Coordinate(0, 0, 0)

    @property
    def e(self) -> "Coordinate":
        return Coordinate(i=self.i - 1, j=self.j - 1, k=self.k)
    
    @property
    def w(self) -> "Coordinate":
        return Coordinate(i=self.i + 1, j=self.j + 1, k=self.k)

    @property
    def ne(self) -> "Coordinate":
        return Coordinate(i=self.i - 1, j=self.j, k=self.k + 1)

    @property
    def nw(self) -> "Coordinate":
        return Coordinate(i=self.i, j=self.j + 1, k=self.k + 1)

    @property
    def se(self) -> "Coordinate":
        return Coordinate(i=self.i, j=self.j - 1, k=self.k - 1)

    @property
    def sw(self) -> "Coordinate":
        return Coordinate(i=self.i + 1, j=self.j, k=self.k - 1)

    def adjacent(self) -> Iterator["Coordinate"]:
        yield self.e
        yield self.w
        yield self.ne
        yield self.nw
        yield self.se
        yield self.sw

    def __add__(self, other: "Coordinate") -> "Coordinate":
        if not isinstance(other, Coordinate):
            raise TypeError("Coordinates must be added to other coordinates")

        return Coordinate(self.i + other.i, self.j + other.j, self.k + other.k)

    def __hash__(self):
        return hash((self.i, self.j, self.k,))

    def __eq__(self, other):
        return isinstance(other, Coordinate) and self.i == other.i and self.j == other.j and self.k == other.k

    def __str__(self) -> str:
        return f"({self.i}, {self.j}, {self.k})"

    def __repr__(self) -> str:
        return f"Coordinate(i={self.i}, j={self.j}, k={self.k})"

In [16]:

class Directions(object):
    def __init__(self, spec: str):
        self.position = Coordinate.zero()

        while spec != "":
            if spec[0] in ["n", "s"]:
                self.position = getattr(self.position, spec[:2])
                spec = spec[2:]
            else:
                self.position = getattr(self.position, spec[0])
                spec = spec[1:]

assert Directions("esew").position == Coordinate(0, -1, -1)
assert Directions("nwwswee").position == Coordinate.zero()

In [29]:
from collections import defaultdict
from typing import List

class Floor(object):
    def __init__(self, flips: List[str]):
        self.black_tiles = set()

        for flip in flips:
            direction = Directions(flip.strip())
            if direction.position in self.black_tiles:
                self.black_tiles.remove(direction.position)
            else:
                self.black_tiles.add(direction.position)

    @property
    def part1(self) -> int:
        return len(self.black_tiles)

    def step(self):
        counts = defaultdict(lambda: 0)

        for position in self.black_tiles:
            for adjacent in position.adjacent():
                counts[adjacent] += 1

        new_black_tiles = set(self.black_tiles)

        for position in self.black_tiles:
            if counts[position] == 0:
                new_black_tiles.remove(position)

        for position, count in counts.items():
            if position in new_black_tiles and (count == 0 or count > 2):
                new_black_tiles.remove(position)
            elif count == 2:
                new_black_tiles.add(position)

        self.black_tiles = new_black_tiles



example = """
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""[1:-1].splitlines()

test_floor = Floor(example)
assert test_floor.part1 == 10

for i in range(1, 11):
    test_floor.step()
    print(f"Day {i}: {test_floor.part1}")

assert test_floor.part1 == 37


Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37


In [30]:
with open("day24.txt", "r") as f:
    data = f.readlines()

floor = Floor(data)
print(f"Total Black Tiles (Part 1): {floor.part1}")

for i in range(0, 100):
    floor.step()

print(f"Total Black Tiles after 100 Days (Part 2): {floor.part1}")

Total Black Tiles (Part 1): 495
Total Black Tiles after 100 Days (Part 2): 4012
